# Embed a holoviews interactive chart in a folium map (0.0.1-alpha)

The output of this notebook can be seen at

https://sfsheath.github.io/holoviews-in-folium/embedded_iframe.html

All very "quick and dirty". Is the an approach that doesn't require writing holoviews variable to a file? I tried using a hv.Renderer object, but the 'raw' html that generates has dependencies that are met when it's written out to an html file.

In [ ]:
import folium
import numpy as np
import pandas as pd
import holoviews as hv
hv.extension('bokeh')

from bokeh.models import HoverTool
from folium import FeatureGroup, LayerControl, Map, Marker
from IPython.display import HTML,IFrame

%matplotlib inline

## Load Data

In [ ]:
ramphs_df = pd.read_csv('http://sebastianheath.com/roman-amphitheaters/roman-amphitheaters.csv')

In [ ]:
# set up df for an index plot
extmajors_df = ramphs_df.sort_values(by = 'extmajor').reset_index()[['index','label','extmajor','latitude','longitude']]

extmajors_df['index'] = extmajors_df.index

extmajors_df.index = extmajors_df.label

extmajors_df.head(2)

## Make the Holoviews Embedable Index Plots

In [ ]:
use_labels = ['Colosseum','Arles','Mactaris'] # choose which amphitheaters to highlight

In [ ]:
hover = HoverTool(tooltips=[("", "<b>@label</b>")])

# An index plot of all know exterior lengths that is the 'background'
# for the individual plots that highlight amphitheaters
index_plot = hv.Points(extmajors_df[['index', 'extmajor','label']].dropna())\
 .opts(plot=dict(height=300, width=400, tools=[hover],color=hv.Cycle('Blues')))

 # loop through 'use_labels' to make a horizontal line showing exterior length of that amphitheater   
maps = {} # this will be a python dictionary that holds all holoviews layouts
for l in use_labels:
    h_line = hv.HLine(extmajors_df.loc[l,'extmajor']).options(color = 'red', line_width= 2, alpha=0.5 )
    
    # 'multiply' background and horizontal line for each amphtitheater, add result to dictionary
    maps[l] = (index_plot * h_line).relabel("Exterior lengths with {} highlighted".format(l))

In [ ]:
# I'd like to skip this step. Can I?? Will keep working on it.
# Problem is writing to files. Not really a problem, but is it necessary??

for l in use_labels:
    hv.util.output('filename="embed_in_folium_{}" fig="html"'.format(l))
    maps[l]

In [ ]:
renderer = hv.renderer("bokeh")
for l in use_labels:
    renderer.save(maps[l], "embed_in_folium_{}".format(l))

## Make Folium map
The content of the iframe is the holoviews viz saved to html.

In [ ]:
m = folium.Map((42, 13), zoom_start=4)


for l in use_labels:
    # get the lat/long of the amphitheater
    latitude = extmajors_df.loc[l,'latitude']
    longitude = extmajors_df.loc[l,'longitude']

    # get the holoviews out as a variable
    with open('embed_in_folium_{}.html'.format(l), 'r') as embed_file:
        html = embed_file.read()

    # create an iframe, then a popup with that iframe as content
    iframe = folium.IFrame(html=html, width=450, height=300)
    popup = folium.Popup(iframe, max_width=2650)

    folium.CircleMarker([latitude, longitude],
                        radius=5,popup=popup,
                        color = "red", fill=True, fill_color = "red").add_to(m)

m.save("embedded_iframe.html", )
m
